In [ ]:
from sklearn import datasets
import pandas as pd

# Load the iris dataset (classification)
iris = datasets.load_iris()
iris_target_name = 'Species'
iris_feature_names = iris.feature_names
iris_df = pd.DataFrame(iris.data, columns=iris_feature_names)
iris_df[iris_target_name] = iris.target

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

# Initialize a spark session
spark = SparkSession.builder.getOrCreate()
df = spark.createDataFrame(iris_df)
df.show(5)

In [ ]:
seed = 123456
df_train, df_test = df.randomSplit([0.7, 0.3], seed=seed)
assembler = VectorAssembler(inputCols=iris_feature_names,
                            outputCol='features')

# Train a Random Forest classification model
rf = RandomForestClassifier().setLabelCol(iris_target_name)
pipe = Pipeline(stages=[assembler, rf])
model = pipe.fit(df_train)
df_pred = model.transform(df_test)
df_pred.show(5)

In [ ]:
from daas_client import DaasClient

# Please, change to your URL of Daas server, and credentials
url = 'https://192.168.64.3:31753'
username = 'admin'
password = 'password'
project = 'Examples'

# Initiate a client of DaaS server, and set the created "Examples" project
client = DaasClient(url, username, password)
if not client.project_exists(project):
    client.create_project(project, 'examples', 'This is an example project')
client.set_project(project)

In [ ]:
from pprint import pprint

model_name = 'spark-cls'

# Publish the built model into DaaS
publish_resp = client.publish(model,
                              name=model_name,
                              mining_function='classification',
                              data_test=df_test,
                              description='A Spark classification model')
pprint(publish_resp)

In [ ]:
# Try to test the published model
test_resp = client.test(model_name, publish_resp['model_version'])
pprint(test_resp)

In [ ]:
# Call the test REST API above, 'model_name' is required in payload because the test runtime serves multiple models 
# in a project. 
import requests

bearer_token = 'Bearer {token}'.format(token=test_resp['access_token'])
payload = {
    'args': {'X': [{'petal length (cm)': 1.5,
                    'petal width (cm)': 0.2,
                    'sepal length (cm)': 4.6,
                    'sepal width (cm)': 3.1}],
             'model_name': model_name,
             'model_version': publish_resp['model_version']}}

response = requests.post(test_resp['endpoint_url'],
                         headers={'Authorization': bearer_token},
                         json=payload,
                         verify=False)
pprint(response.json())

In [ ]:
# Deploy the published model into product
deploy_resp = client.deploy(model_name, 
                            deployment_name=model_name + '-svc', 
                            model_version=publish_resp['model_version'])
pprint(deploy_resp)

In [ ]:
# Call the product REST API above, the deployment runtime(s) serve the deployed model dedicatedly.
deploy_bearer_token = 'Bearer {token}'.format(token=deploy_resp['access_token'])
deploy_payload = {
    'args': {'X': [{'petal length (cm)': 1.5,
                    'petal width (cm)': 0.2,
                    'sepal length (cm)': 4.6,
                    'sepal width (cm)': 3.1}]}}

response = requests.post(deploy_resp['endpoint_url'],
                         headers={'Authorization': deploy_bearer_token},
                         json=deploy_payload,
                         verify=False)
pprint(response.json())